### `CASE NAVEGAÇÂO INTERIOR - BRASHIROVE`

Mapa esquemático representando as rotas & modais constantes no enunciado, com as informações mais relevantes.

In [1]:
import geopandas as gpd
import folium

#### `>>>> Leitura dos dados`

In [2]:
# hidrovias
gdf_aqa = gpd.read_file("../data/vw_snv_aqa.json")

# Portos das hidrovias
gdf_portos = gpd.read_file("../data/vw_snv_portos.json")

# Rodovias
gdf_cide_rod = gpd.read_file("../data/vw_cide_rod.json", driver="GeoJSON")

# Ferrovias
gdf_dif_ferrovias = gpd.read_file("../data/vw_dif_ferrovias.json", driver="GeoJSON")
gdf_fns = gpd.read_file("../data/fns.json", driver="GeoJSON")

In [3]:

# Rota 1 concorrentes
gdf_r1c = gpd.read_file("../data/mygeodata/mineiros-suape-rodo-2023-12-13-2739km/tracks.geojson", driver="GeoJSON")

#### `>>> Filtra as hidrovias`

In [4]:
gdf_hv = gdf_aqa[gdf_aqa["Nome"].str.contains("francisco", case=False)]
gdf_hv

,id,id_snv,Nome,Codigo,Sequencia_SNV,Trecho,Extensao_km,Administracao,Regiao_Hidrografica,dt_ini,dt_fim,geometry
46,vw_snv_aqa.fid-10762dc3_18ab49b5b54_5f18,46,Rio S. Francisco,HF-500,0150,"Da foz, no oceano Atlantico, até Iguatama (MG)",2604.7,AHSFRA,Sao Francisco,2013-01-01,2029-10-30,MULTILINESTRING Z ((-40.49683 -9.40821 0.00000...


#### `>>> Fitra os portos`

In [5]:
mask1 = gdf_portos["Local"].str.contains("francisco", case=False)
mask2 = gdf_portos["Nome"].isin(["Petrolina", "Pirapora"])
gdf_portos_sf = gdf_portos[mask1 & mask2]

gdf_portos_sf

,id,id_porto_snv,Nome,Unidade_Federacao,Local,lat,long,dt_ini,dt_fim,geometry
42,vw_snv_portos.fid-10762dc3_18ab5149929_28e1,43,Petrolina,PE,Rio São Francisco,-9.4037,-4.0500,2013-01-01,2029-10-30,MULTIPOINT (-40.50063 -9.40370)
100,vw_snv_portos.fid-10762dc3_18ab5149929_291b,101,Pirapora,MG,Rio São Francisco,-1.7338,-4.4954,2013-01-01,2029-10-30,MULTIPOINT (-44.95416 -17.33895)


In [6]:
from shapely.geometry import Point

# cria do gdf dos pontos dos terminais
gdf = {
    "Nome": ["Porto Petrolina", "Porto Pirapora"],
    "UF": ["PE", "MG"],
    "geometry": [
        Point(-40.50063, -9.40370),
        Point(-44.95416, -17.33895),
    ],
}

gdf_hv_portos = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf_hv_portos

gdf = {
    "Nome": ["Porto Pecem", "Porto Suape"],
    "UF": ["CE", "PE"],
    "geometry": [
        Point(-38.811599, -3.546613),
        Point(-34.970137, -8.395096),
    ],
}

gdf_mr_portos = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf_mr_portos

,Nome,UF,geometry
0,Porto Pecem,CE,POINT (-38.81160 -3.54661)
1,Porto Suape,PE,POINT (-34.97014 -8.39510)


#### `>>> Filtra os termináis ferroviários`


In [7]:

gdf_termi = gpd.read_file("../data/vw_epl_terminais.json", driver="GeoJSON")

mask1 = gdf_termi["Unidade_Federacao"].isin(["GO"])
# mask2 = gdf_termi["id_terminal"].isin([615, 621, 622, 623, 624, 625, 708, 709, 710, 711, 727])
mask2 = gdf_termi["id_terminal"].isin([615, 621])
gdf_termi_ns = gdf_termi[mask1 & mask2]
# gdf_termi_ns = gdf_termi[mask1]["id_terminal"].values
gdf_termi_ns;

In [8]:
from shapely.geometry import Point

# cria do gdf dos pontos dos terminais
gdf = {
    "Nome": [
        "Anápolis", "Paracatu", "Luiziânia do Oeste", "Recife",
        "Estreito", "Eliseu Martins", "Salgueiro", "Corinto", "Propriá"
    ],
    "UF": ["GO", "MG", "MG", "PE", "MA", "PI", "PE", "MG", "AL"],
    "Tipo": ["Terminal Ferroviário"] * 9,
    "geometry": [
        Point(-48.918045, -16.403519),
        Point(-46.872679, -17.235817),
        Point(-45.626183, -17.938339),
        Point(-34.931877, -8.085505),
        Point(-47.4538, -6.5569),  # Estreito/MA
        Point(-43.6641, -8.0981),  # Eliseu Martins/PI
        Point(-39.12541, -8.07500),  # Salgueiro/PE
        Point(-44.4557, -18.3630),  # Corinto/MG
        Point(-36.82875, -10.22046),  # Propriá/AL
    ],
}
gdf_fv_terminais = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf_fv_terminais

gdf = {
    "Nome": ["Rio Verde", "Mineiros"],
    "UF": ["GO", "GO"],
    "Tipo": ["Produção Agrícola"] * 2,
    "geometry": [
        Point(-50.928125, -17.788950),
        Point(-52.574080, -17.550768),
    ],
}
gdf_fv_prod = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf_fv_prod

,Nome,UF,Tipo,geometry
0,Rio Verde,GO,Produção Agrícola,POINT (-50.92813 -17.78895)
1,Mineiros,GO,Produção Agrícola,POINT (-52.57408 -17.55077)


#### `>>> Filtra as ferrovias`

In [9]:
# Ferrovias Pecem
mask1 = gdf_dif_ferrovias["Unidade_Federacao"].isin(["CE"])
mask2 = gdf_dif_ferrovias["id_ferrovia"].isin([
    846, 842, 840, 760, 755, 841, 831, 761, 737, 768, 746, 745, 739, 764, 756, 752, 769, 736, 762,
    712, 754, 751, 896, 763, 744, 747, 759, 743, 748, 716, 750, 753, 749,770
])
gdf_fv_pecem = gdf_dif_ferrovias[mask1 & ~mask2]
# gdf_fv_pecem[gdf_fv_pecem["id_ferrovia"] == 676]["geometry"]
gdf_fv_pecem;

# Ferrovias Suape
mask1 = gdf_dif_ferrovias["Unidade_Federacao"].isin(["PE"])
mask2 = gdf_dif_ferrovias["id_ferrovia"].isin([
524, 788, 796, 799, 779, 775, 795, 663, 655, 664, 771, 661, 862, 857, 885, 865, 877, 856, 887,
879, 884, 868, 875, 860, 882, 869, 858, 660, 662, 787, 784, 659
])
gdf_fv_suape = gdf_dif_ferrovias[mask1 & ~mask2]
# gdf_fv_suape[gdf_fv_suape["id_ferrovia"] == 676]["geometry"]
gdf_fv_suape;

In [37]:
# Ferrovia Norte-Sul

mask1 = gdf_dif_ferrovias["Unidade_Federacao"].isin(["GO"])
mask2 = gdf_dif_ferrovias["id_ferrovia"].isin([38])
# mask2 = gdf_dif_ferrovias["Concessionaria"].isin(["FNS"])
gdf_fv_go = gdf_dif_ferrovias[mask1 & mask2]
# gdf_fv_go = gdf_dif_ferrovias[mask1]
# gdf_fv_go[gdf_fv_go["id_ferrovia"] == 676]["geometry"]
gdf_fv_go;

mask1 = gdf_fns['geometry'].apply(lambda geom: isinstance(geom, Point))
gdf_fns = gdf_fns[~mask1]
gdf_fns = gdf_fns[["geometry"]]
gdf_fns;

In [38]:
# Ferrovia FCA (Pirapora - Salvador - Petrolina)

mask1 = gdf_dif_ferrovias["Unidade_Federacao"].isin(["MG", "BA", "SE"])
mask2 = gdf_dif_ferrovias["id_ferrovia"].isin([
    195, 196, 198, 199, 359, 358, 357, 356, 354, 353, 352, 351, 430, 350, 349, 348, 254, 347, 346,
    345, 344, 2076, 139, 123, 202, 203, 253, 34, 204, 129, 264, 263, 261, 260, 259, 258, 257, 256, 255,
    252, 251, 250, 249, 248, 247, 246,

])
mask3 = gdf_dif_ferrovias["Concessionaria"].isin(["FCA"])
substrings = [
    "Uberaba", "Araguari", "Angra", "Araxá", "Caldas", "Roncador", "Divinópolis",
    "Ponte Nova", "Recreio", "Três", "Caratinga", "Recreio", "Calafate", "Pedreira", "Burnier",
    "Eldorado", "Sabará"
]
pattern = '|'.join(substrings)
mask4 = gdf_dif_ferrovias["Linha"].str.contains(pattern, case=False)
gdf_fv_fca = gdf_dif_ferrovias[mask1 & ~mask2 & mask3 & ~mask4]
# gdf_fv_fca = gdf_dif_ferrovias[mask2]
# gdf_fv_go[gdf_fv_go["id_ferrovia"] == 676]["geometry"]
# gdf_fv_fca

In [39]:
# Ferrovia FTL (Pirapora - Propriá - Recife)

mask1 = gdf_dif_ferrovias["Unidade_Federacao"].isin(["SE", "AL", "PE"])
mask2 = gdf_dif_ferrovias["Concessionaria"].isin(["FCA", "FTL"])
substrings = [
    "Uberaba", "Araguari", "Angra", "Araxá", "Caldas", "Roncador", "Garças", "Divinópolis",
    "Ponte Nova", "Recreio", "Três", "Caratinga", "Recreio", "Calafate", "Pedreira", "Burnier",
    "Eldorado", "Sabará", "Tronco Norte", "Tronco Centro", "Grande Recife", "Boa Viagem", "Propriá"
]
pattern = '|'.join(substrings)
mask4 = gdf_dif_ferrovias["Linha"].str.contains(pattern, case=False)
gdf_fv_ftl = gdf_dif_ferrovias[mask1 & mask2 & ~mask4]
# gdf_fv_ftl

In [40]:
from shapely.geometry import LineString
import pandas as pd

gdf = {}
gdf["Nome"] = ["Ferrovia Anápolis-Pirapora"]
gdf["geometry"] = [
    LineString([
        (-48.918045, -16.403519),
        (-46.872679, -17.235817),
        (-45.626183, -17.938339),
        (-44.95416, -17.33895),
    ])
]
gdf_fv_anapolis_pirapora = gpd.GeoDataFrame(gdf, crs="EPSG:4326")

gdf = gdf_fv_pecem[["geometry"]].to_crs("EPSG:4326")
gdf2 = gpd.GeoDataFrame(
    {
        "geometry": [LineString([
            (-39.40762247,-7.23259637),
            (-40.50063, -9.40370),
        ])]
    },
    crs="EPSG:4326",
)
gdf_fv_petrolina_pecem = pd.concat([gdf, gdf2])
gdf_fv_petrolina_pecem["Nome"] = "Ferrovia Petrolina-Pecem"

gdf = gdf_fv_suape[["geometry"]].to_crs("EPSG:4326")
gdf2 = gpd.GeoDataFrame(
    {
        "geometry": [
            LineString([
                (-40.50063, -9.40370),
                (-39.12542462,-8.07463528),
            ]),
            LineString([
                (-35.0053399,-8.36461191),
                (-35.00511701,-8.36199339),
                (-35.00114742,-8.34836379),
                (-35.00743451,-8.337292),
                (-35.00456772,-8.3194359),
                (-35.00539383,-8.2934897),
                (-34.99950685,-8.2711111),
                (-34.99747072,-8.26977236),
            ])
        ]
    },
    crs="EPSG:4326",
)
gdf_fv_petrolina_suape = pd.concat([gdf, gdf2])
gdf_fv_petrolina_suape["Nome"] = "Ferrovia Petrolina-Suape"

In [41]:
from shapely.geometry import LineString
import pandas as pd

gdf = {}
gdf["Nome"] = ["Ferrovia Estreito/MA - Salgueiro/PE"]
gdf["geometry"] = [
    LineString([
        (-47.4538, -6.5569),  # Estreito/MA
        (-43.6641, -8.0981),  # Eliseu Martins/PI
        (-41.1431, -8.1343),  # Paulistana/PI
        (-39.12541, -8.07500),  # Salgueiro/PE
    ])
]
gdf_fv_tlsa = gpd.GeoDataFrame(gdf, crs="EPSG:4326")

#### `>>> Filtra as rodovias estaduais`

In [42]:
mask1 = gdf_cide_rod["Unidade_Federacao"].isin(["GO"])
mask2 = gdf_cide_rod["Codigo_Rodovia"].isin(["164"])
# mask2 = gdf_cide_rod["Concessionaria"].isin(["FNS"])
gdf_rd_go = gdf_cide_rod[mask1 & mask2]
# # gdf_rd_go[gdf_rd_go["id_ferrovia"] == 676]["geometry"]
gdf_rd_go;

In [43]:
from shapely.geometry import LineString
import pandas as pd

gdf = {}
gdf["Nome"] = ["Caminho Produção Agricola"]  # * 2
gdf["geometry"] = [
    # LineString([
    #     (-52.574080, -17.550768),
    #     (-50.578338, -17.792409),
    # ]),
    LineString([
        (-50.928125, -17.788950),
        (-50.578338, -17.792409),
    ])
]
gdf_prod_go = gpd.GeoDataFrame(gdf, crs="EPSG:4326")

### `>>> Styles`

In [51]:
style_fv = lambda x: {
    "color":"brown",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_petrolina_suape = lambda x: {
    "color":"pink",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_ns = lambda x: {
    "color":"orange",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_go164 = lambda x: {
    "color":"grey",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_caminho_pagric = lambda x: {
    "color":"green",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_fca = lambda x: {
    "color":"purple",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_ftl = lambda x: {
    "color":"yellow",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

style_fv_es = lambda x: {
    "color":"lightgreen",
    "dashArray": "5, 5",
    "weight": 5,
    "stroke": True,
    "fill": False,
}

### `>>> Map`

In [52]:

m = folium.Map(location=[-10,-50], zoom_start=6)

folium.GeoJson(
    gdf_hv_portos,
    name="Portos Fluviais",
    marker=folium.Marker(icon=folium.Icon(color="blue", icon="anchor", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["Nome"]),
    popup=folium.GeoJsonPopup(fields=["Nome"]),
).add_to(m)
folium.GeoJson(
    gdf_hv,
    name="Hidrovia São Francisco",
    tooltip=folium.GeoJsonTooltip(fields=["Nome"]),
    popup=folium.GeoJsonPopup(fields=["Nome"]),
).add_to(m)
folium.GeoJson(
    gdf_fv_terminais,
    name="Terminais Ferroviarios",
    marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["Nome", "Tipo"]),
    popup=folium.GeoJsonPopup(fields=["Nome", "Tipo"]),
).add_to(m)
folium.GeoJson(
    gdf_fv_prod,
    name="Produção Agrícola",
    marker=folium.Marker(icon=folium.Icon(color="green", icon="tree", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["Nome", "Tipo"]),
    popup=folium.GeoJsonPopup(fields=["Nome", "Tipo"]),
).add_to(m)
folium.GeoJson(
    gdf_fv_anapolis_pirapora,
    name="Ferrovia Anápolis-Pirapora",
    marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["Nome"]),
    popup=folium.GeoJsonPopup(fields=["Nome"]),
    style_function=style_fv,
).add_to(m)
folium.GeoJson(
    gdf_fv_petrolina_suape,
    name="Ferrovia Petrolina-Suape",
    tooltip=folium.GeoJsonTooltip(fields=["Nome"]),
    popup=folium.GeoJsonPopup(fields=["Nome"]),
    style_function=style_fv_petrolina_suape,
).add_to(m)

folium.GeoJson(
    gdf_fv_go,
    name="Ferrovia Norte-Sul",
#     marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
#     tooltip=folium.GeoJsonTooltip(fields=["id_ferrovia", "Concessionaria"]),
#     popup=folium.GeoJsonPopup(fields=["id_ferrovia", "Concessionaria"]),
    style_function=style_fv_ns,
).add_to(m)

folium.GeoJson(
    gdf_fns,
    name="Ferrovia Norte-Sul",
    marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    # tooltip=folium.GeoJsonTooltip(fields=["id_ferrovia", "Concessionaria"]),
    # popup=folium.GeoJsonPopup(fields=["id_ferrovia", "Concessionaria"]),
    style_function=style_fv_ns,
).add_to(m)

folium.GeoJson(
    gdf_prod_go,
    name="Caminha Produção Agricola",
    marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["Nome"]),
    popup=folium.GeoJsonPopup(fields=["Nome"]),
    style_function=style_fv_caminho_pagric,
).add_to(m)

folium.GeoJson(
    gdf_rd_go,
    name="Rodovia GO-164",
    # tooltip=folium.GeoJsonTooltip(fields=["Codigo_Rodovia"]),
    # popup=folium.GeoJsonPopup(fields=["Codigo_Rodovia"]),
    style_function=style_fv_go164,
).add_to(m)

# Rota 1 - Rodoviario
folium.GeoJson(
    gdf_r1c,
    name="Rota rodoviaria Mineiros-Suape",
#     # tooltip=folium.GeoJsonTooltip(fields=["Nome"]),
#     # popup=folium.GeoJsonPopup(fields=["Nome"]),
    style_function=style_fv_go164,
).add_to(m)

# Rota 2 - Rodo + Ferro (Pirapora - Salvador - Petrolina)
folium.GeoJson(
    gdf_fv_fca,
    name="Ferrovia FCA",
    # marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["id_ferrovia", "Concessionaria", "Linha"]),
    # popup=folium.GeoJsonPopup(fields=["id_ferrovia", "Concessionaria"]),
    style_function=style_fv_fca,
).add_to(m)

# Rota 3 - Rodo + Ferro (Pirapora - Propiá - Recife)
folium.GeoJson(
    gdf_fv_ftl,
    name="Ferrovia FTL",
    # marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    tooltip=folium.GeoJsonTooltip(fields=["id_ferrovia", "Concessionaria", "Linha"]),
    # popup=folium.GeoJsonPopup(fields=["id_ferrovia", "Concessionaria"]),
    style_function=style_fv_ftl,
).add_to(m)

# Rota 4 - Rodo + Ferro (Santa Helena de Goiás - Estreito - Salgueiro - Recife)
folium.GeoJson(
    gdf_fv_tlsa,
    name="Ferrovia TLSA",
    # marker=folium.Marker(icon=folium.Icon(color="red", icon="train", prefix="fa")),
    # tooltip=folium.GeoJsonTooltip(fields=["id_ferrovia", "Concessionaria", "Linha"]),
    # popup=folium.GeoJsonPopup(fields=["id_ferrovia", "Concessionaria"]),
    style_function=style_fv_es,
).add_to(m)

m.fit_bounds(m.get_bounds())
m.save("mapa-brashirove-v4.html")
m